# Capstone Project-Coffeeshops/Diners in Harlem NYC


In [1]:
!pip install folium
import pandas as pd
import folium


In [6]:
import requests

In [7]:
request_parameters = {
    "client_id": 'CLIENT_ID',
    "client_secret": 'CLIENT_SECRET',
    "v": '20190111',
    "section": "coffee",
    "near": "Harlem,NY",
    "limit": 50}

In [8]:
data=requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [9]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [10]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Harlem', 'Harlem')

In [11]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 40.83037859085295, 'lng': -73.92819198456822},
  'sw': {'lat': 40.791114741664096, 'lng': -73.96735953829558}},
 95)

In [12]:
d["geocode"]

{'what': '',
 'where': 'harlem ny',
 'center': {'lat': 40.80788, 'lng': -73.94542},
 'displayString': 'Harlem, NY, United States',
 'cc': 'US',
 'geometry': {'bounds': {'ne': {'lat': 40.8308258057, 'lng': -73.9284744263},
   'sw': {'lat': 40.7925949097, 'lng': -73.9655838013}}},
 'longId': '72057594043047970'}

In [13]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [14]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '537f7140498e15919e85ff84',
  'name': 'Manhattanville Coffee',
  'location': {'address': '142 Edgecombe Ave',
   'crossStreet': 'at W 142nd St',
   'lat': 40.8214960193694,
   'lng': -73.94459457496639,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.8214960193694,
     'lng': -73.94459457496639},
    {'label': 'entrance', 'lat': 40.821315, 'lng': -73.944578}],
   'postalCode': '10030',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['142 Edgecombe Ave (at W 142nd St)',
    'New York, NY 10030',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': 

In [16]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i restaurants" % len(df))
df.head()

#First 5 results

found 48 restaurants


,uid,name,shortname,address,postalcode,lat,lng
0,537f7140498e15919e85ff84,Manhattanville Coffee,Coffee Shop,283 Pleasant Ave,10030,40.821496,-73.944595
1,541c683e498ef421d4e16eb3,The Edge Harlem,American,283 Pleasant Ave,10030,40.819692,-73.946073
2,52e3eef9498e1affb602399b,Double Dutch Espresso,Coffee Shop,283 Pleasant Ave,10026,40.805893,-73.954222
3,4af2e1aaf964a520f2e821e3,The Love Cafe,Café,283 Pleasant Ave,10029,40.794550,-73.933798
4,4eb0d7bc4fc6015c82ea1c3e,Lenox Coffee,Coffee Shop,283 Pleasant Ave,10027,40.810161,-73.943247


In [14]:
#Folium Map Code
import folium
from folium import plugins

map_Harlem = folium.Map(location=[40.80788, -73.94542], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Harlem)
map_Harlem